In [30]:
# libraries
import trep
from trep import tx,ty,tz,rx,ry,rz
import trep.potentials
import time
import trep.visual as visual
import math
import numpy as np
import csv

In [1]:
# import states from .csv

# make sure .csv has ['motor1', 'motor2', 'motor3'] 
# or trep model's q = q = [z_com, motor1, center_theta1, center_theta2, motor2, right_theta, motor3, left_theta]

def getStates(filename):
    
    # read states from csv
    states = []
    with open(filename, 'r') as csvfile:
        csvReader = csv.reader(csvfile)
        for row in csvReader:
            float_row = []
            for state in row:
                float_row.append(float(state))
            states.append(float_row)
        
    # detect and print the format (3 motor angles or all trep angles)    
    print np.shape(states)
    if (np.shape(states)[1]==3):
        print 'Only motor torques'
    else:
        print 'States for trep model'
    
    return states

filename = 'JointVelocities.csv'
states = getStates(filename)

print states

NameError: global name 'csv' is not defined

In [17]:
# OR calculate states with forward kinematics

def geomFK(qa,lengths,solOption):
    
    # Description:
    #    Computes the foot pose, using the actuated joint positions.
    #    Uses geometry (intersecting circles) to solve for the locations of the 
    #    "ankle" (xA,yA) and the "upper ankle" (xuA, yuA) relative to the base 
    #    frame, then calculates unactuated joint positions and the foot pose.

    #  References:
    #    https://math.stackexchange.com/questions/256100/how-can-i-find-the-points-at-which-two-circles-intersect
    #    http://mathworld.wolfram.com/Circle-CircleIntersection.html

    #  Inputs:
    #    qa:     actuated joint positions (column vector)
    #            qa = [theta1; phi1; psi1];
    #    lengths: the relevant dimensions of the robot, stored in a 10x1 column
    #        vector according to this order:
    #        lengths = [L1; L2; L3; L4; L5 L6; L7; L8; B1x; B2x; B1y; B2y], where
    #        B1x, B1y, L1, L2, and L8 correspond to the theta-chain,
    #        L3, L4, L7, and L8 correspond to the phi-chain, and
    #        B2x, B2y, L5, L6, and L8 correspond to the psi-chain.
    #    solOption: 1, 2, 3, or 4
    #        1: function uses (xA1,yA1) and (xuA1, yuA1)
    #        2: function uses (xA1,yA1) and (xuA2, yuA2)
    #        3: function uses (xA2,yA2) and (xuA1, yuA1)
    #        4: function uses (xA2,yA2) and (xuA2, yuA2)

    #  Outputs:
    #    qu:         6x1 column vector
    #        qu = [th2; th3; ph2; ph3; ps2; ps3];
    #    footPose:   3-element row vector which comprises:
    #        xF:     x-coordinate of the foot, relative to the body
    #        yF:     y-coordinate of the foot, relative to the body
    #        angF:   angle of the foot, relative to the body.

    #  Dependencies:
    #    none

    # "unpack" link lengths from "lengths" input vector:
    # Note: this step is just for cleaner-looking code.
    
    # make our own wrap to pi function
    def wrapToPi(phases):
        return ((phases+np.pi)%(2*np.pi)) - np.pi
        
    L1 = lengths[0]
    L2 = lengths[1]
    L3 = lengths[2]
    L4 = lengths[3]
    L5 = lengths[4]
    L6 = lengths[5]
    L7 = lengths[6]
    L8 = lengths[7]
    B1x = lengths[8]
    B2x = lengths[9]
    B1y = lengths[10]
    B2y = lengths[11]

    #  "unpack" actuated joint angles from "qa" input vector:
    #  Note: this step is just for cleaner-looking code.
    th1 = qa[0]
    ph1 = qa[1]
    ps1 = qa[2]

    # create empty arrays:
    qu = np.zeros((6,1))
    footPose = np.zeros((3,1))
    
    # initialize pi
    pi = math.pi

    # Calculate (xA, yA) using th1 and ps1:
    xkth = -B1x + L1*math.cos(th1)  #x-coordinate of theta-chain "knee"
    ykth = B1y + L1*math.sin(th1)   # y-coordinate of theta-chain "knee"

    xkps = B2x + L5*math.cos(ps1)  # x-coordinate of psi-chain "knee"
    ykps = B2y + L5*math.sin(ps1)  # y-coordinate of psi-chain "knee"

    a = math.sqrt((xkth - xkps)**2 + (ykth - ykps)**2)    # intermediate variable
    b = (L2**2 - L6**2 + a**2)/(2*a)                  # intermediate variable
    c = math.sqrt(L2**2 - b**2)                           # intermediate variable

    xA1 = (b/a)*(xkps - xkth) + (c/a)*(ykps - ykth) + xkth   # solution 1/2 xA-coordinate
    xA2 = (b/a)*(xkps - xkth) - (c/a)*(ykps - ykth) + xkth   # solution 3/4 xA-coordinate

    yA1 = (b/a)*(ykps - ykth) - (c/a)*(xkps - xkth) + ykth   # solution 1/2 yA-coordinate
    yA2 = (b/a)*(ykps - ykth) + (c/a)*(xkps - xkth) + ykth   # solution 3/4 yA-coordinate
    
    if ( (solOption==1) | (solOption==2) ):
        xA = xA1
        yA = yA1
    else:
        xA = xA2
        yA = yA2
    
    # fprintf('(xA, yA) = (%f, %f)\n',[xA, yA]);

    # Calculate (xuA, yuA) using ph1 and (xA, yA):
    xkph = L3*math.cos(ph1)  # x-coordinate of phi-chain "knee"
    ykph = L3*math.sin(ph1)   # y-coordinate of phi-chain "knee"
    
    d = math.sqrt((xkph - xA)**2 + (ykph - yA)**2)    # intermediate variable
    e = (L4**2 - L7**2 + d**2)/(2*d)           # intermediate variable
    f = math.sqrt(L4**2 - e**2)                       # intermediate variable

    xuA1 = (e/d)*(xA - xkph) - (f/d)*(yA - ykph) + xkph   # solution 1/3 xuA-coordinate
    xuA2 = (e/d)*(xA - xkph) + (f/d)*(yA - ykph) + xkph   # solution 2/4 xuA-coordinate

    yuA1 = (e/d)*(yA - ykph) + (f/d)*(xA - xkph) + ykph   # solution 1/3 xuA-coordinate
    yuA2 = (e/d)*(yA - ykph) - (f/d)*(xA - xkph) + ykph   # solution 2/4 xuA-coordinate

    if ( (solOption==1) | (solOption==3) ):
        xuA = xuA1
        yuA = yuA1
    else:
        xuA = xuA2
        yuA = yuA2
    
    # fprintf('(xuA, yuA) = (%f, %f)\n',[xuA, yuA]);

    # Solve for "knee" angles (th2, ph2, ps2):

    # theta-chain--------------------------------------------------------------
    # Calculate (x,y) location of theta-chain "hip" joint:
    xHtheta = -B1x
    yHtheta = B1y

    # ankle relative to theta-hip:
    xAptheta = -xHtheta + xA
    yAptheta = yHtheta - yA

    # Calculate knee angle:
    betatheta = wrapToPi(math.acos((L1**2 + L2**2 - xAptheta**2 - yAptheta**2)/(2*L1*L2)))
    th2 = wrapToPi(pi - betatheta)

    # fprintf('theta2 = %f degrees\n',th2*(180/pi));

    # save th2 to qu array:
    qu[0] = th2

    # phi-chain----------------------------------------------------------------
    # Calculate knee angle:
    betaphi = wrapToPi(math.acos((L3**2 + L4**2 - xuA**2 - yuA**2)/(2*L3*L4)))
    ph2 = wrapToPi(pi - betaphi)

    # fprintf('phi2 = %f degrees\n',ph2*(180/pi));

    # save ph2 to qu array:
    qu[2] = ph2

    # psi-chain----------------------------------------------------------------
    # Calculate (x,y) location of "hip" joint:
    xHpsi = B2x
    yHpsi = B2y

    # Calculate hip angle:
    xAppsi = xHpsi - xA
    yAppsi = yHpsi - yA

    # Calculate knee angle:
    betapsi = wrapToPi(math.acos((L5**2 + L6**2 - xAppsi**2 - yAppsi**2)/(2*L5*L6)))
    ps2 = wrapToPi(pi + betapsi)

    # fprintf('psi2 = %f degrees\n',ps2*(180/pi));

    # save ps2 to qu array:
    qu[4] = ps2

    # Calculate the foot pose:
    footAngle = wrapToPi(pi+ math.atan2(yuA - yA, xuA - xA))
    footX = xA + L8*math.cos(footAngle)
    footY = yA + L8*math.sin(footAngle)

    # "pack" foot pose variables into footPose:
    footPose[0] = footX
    footPose[1] = footY
    footPose[2] = footAngle

    # Calculate the third angle in each open chain:
    th3 = footAngle - th1 - th2
    ph3 = footAngle - ph1 - ph2
    ps3 = footAngle - ps1 - ps2

    qu[1] = th3
    qu[3] = ph3
    qu[5] = ps3
    
    # TODO: calculate angles for trep model input here!
    # qu = [th2, th3, ph2, ph3, ps2, ps3] = [center_theta1, center_theta2, right_theta, ~, left_theta, ~]
    qtrep = [0,0,th2,th3,0,ph2,0,ps2]
    
    return (qu, qtrep, footPose)


# interpolated motor angles
motor_angles = [qnew[1]-(np.pi/2), qnew[4]-(np.pi/2), qnew[6]-(np.pi/2)]
# motor_angles = [-2.43, -1.303, -1.341]
print motor_angles
print '\n'


# real link lengths, lengths = [L1; L2; L3; L4; L5 L6; L7; L8; B1x; B2x; B1y; B2y] (from google doc)
lengths = [0.053, 0.139, 0.097, 0.0983, 0.053, 0.139, 0.0692, 0.1018, 0.0573, 0.0573, 0.0082, 0.0082]


# calculate the other angles and reorder for trep
(qu, qtrep, footPose) = geomFK(motor_angles, lengths, 1)

print footPose
print '\n'
print qu
print '\n'
print qtrep
# print footPose
print'\n'
print qnew
print '\n'

(qu, qtrep, footPose) = geomFK(motor_angles, lengths, 2)

print footPose
print '\n'
print qu
print '\n'
print qtrep
# print footPose
print'\n'
print qnew
print '\n'

[-2.4279117225431306, -1.3035348854812348, -1.3408180802736125]


[[-0.10047337]
 [-0.07676716]
 [ 2.40329283]]


[[ 1.40316693]
 [ 3.42803763]
 [ 0.2633574 ]
 [ 3.44347032]
 [-0.97815665]
 [ 4.72226756]]


[0, 0, 1.4031669260625943, 3.4280376263271002, 0, 0.26335739990387985, 0, -0.9781566480353412]


[ 0.37754284 -0.8571154   1.96966489 -1.19988333  0.26726144 -0.72479644
  0.22997825  0.16173713]


[[ 0.04468557]
 [-0.21932193]
 [-0.81439449]]


[[ 1.40316693]
 [ 0.21035031]
 [ 1.82385979]
 [-1.3347194 ]
 [-0.97815665]
 [ 1.50458024]]


[0, 0, 1.4031669260625943, 0.21035030836688007, 0, 1.8238597945803292, 0, -0.9781566480353412]


[ 0.37754284 -0.8571154   1.96966489 -1.19988333  0.26726144 -0.72479644
  0.22997825  0.16173713]




In [45]:
# set up robot model
hopper = trep.System()
hopper.import_frames([
    # center of mass
    tz('z_com',mass=0.001,name='CoM_robot'), [
        # middle links
        tz(-0.1), [
            rx('motor1',name='motor1',kinematic=True), [
                tz(-0.097,name='center_link1'), [
                    rx('center_theta1'), [
                        tz(-0.0983,name='center_link2'), [
                            rx('center_theta2'), [
                                tz(-0.0692,mass=0.001,name='center_attach'), [
                                    tz(-0.1018,name='foot')
                                ]
                            ]
                        ]
                    ]
                ]
            ]
        ],
        # right links
        ty(0.0573), [
            tz(-0.0918), [
                rx('motor2',name='motor2',kinematic=True), [
                    tz(-0.053,name='right_link'), [
                        rx('right_theta'), [
                            tz(-0.139,name='right_attach')
                        ]
                    ]
                ]
            ]
        ],
        # left links
        ty(-0.0573), [
            tz(-0.0918), [
                rx('motor3',name='motor3',kinematic=True), [
                    tz(-0.053,name='left_link'), [
                        rx('left_theta'), [
                            tz(-0.139,name='left_attach')
                        ]
                    ]
                ]
            ]
        ]
    ]
])

# NOTE: q = [z_com, motor1, center_theta1, center_theta2, motor2, right_theta, motor3, left_theta]

# Establish gravity
trep.potentials.Gravity(hopper)
# trep.forces.Damping(system, 0.1)

# Input Torque
trep.forces.ConfigForce(hopper, 'motor1', 'motor1_torque')
trep.forces.ConfigForce(hopper, 'motor2', 'motor2_torque')
trep.forces.ConfigForce(hopper, 'motor3', 'motor3_torque')

# Add constraints
trep.constraints.PointToPoint2D(hopper,'yz','right_attach','center_attach')
trep.constraints.PointToPoint2D(hopper,'yz','left_attach','center_attach')
# trep.constraints.PointToPoint2D(hopper,'yz','World','foot')
# constrain foot angle to vertical
# constrain outside angles to be symmetric
# constrain all three angles to their real ranges

print hopper.nQ
print hopper.nQd
print hopper.nQk

8
5
3


In [3]:
# simulate forward with trajectory

In [1]:
# OR simulate forward with force input

In [47]:
# visualize results

# initial configuration, # q = [z_com, motor1, center_theta1, center_theta2, motor2, right_theta, motor3, left_theta]
qinit = [0.4, 0.1, 0.5, -0.5, 0.6, -0.2, -0.6, 0.2]


motors = [-2.4279117225431306, -1.3035348854812348, -1.3408180802736125]
qFK1 = [0.4, motors[0]+(np.pi/2), 1.4031669260625943, 3.4280376263271002, motors[1]+(np.pi/2), 0.26335739990387985, motors[2]+(np.pi/2), -0.9781566480353412]
qFK2 = [0.4, motors[0]+(np.pi/2), 1.4031669260625943, 3.4280376263271002, motors[1]+(np.pi/2), 0.26335739990387985, motors[2]+(np.pi/2), -0.9781566480353412]


hopper.q = qinit
hopper.satisfy_constraints(tolerance=1e-1)
qnew = hopper.q
print qnew
print '\n'

hopper.q = qFK1
hopper.satisfy_constraints(tolerance=1e-1)
qnew1 = hopper.q
print qnew1
print '\n'

hopper.q = qFK2
hopper.satisfy_constraints(tolerance=1e-1)
qnew2 = hopper.q
print qnew2
print '\n'

q = np.repeat(qnew1, 101, axis=0).reshape((8,101)).T
t = np.linspace(0,5,101)

# Display
# print "Simulation: dt=%f, tf=%f, runtime=%f s" % (dt, tf, finish-start)
visual.visualize_3d([ visual.VisualItem3D(hopper, t, q) ],camera_pos = [(1,0,0)])

[ 0.4         0.78559714  1.46519329 -0.19709016  0.35624717 -0.80172297
 -0.17052417  0.02516772]


[ 0.4        -1.01046995  2.6288954   4.89147916  0.93527714  0.27346489
  0.8562392  -0.19232566]


[ 0.4        -1.01046995  2.6288954   4.89147916  0.93527714  0.27346489
  0.8562392  -0.19232566]




0